In [11]:
import os
import cv2
import numpy as np

from skimage.feature import local_binary_pattern, hog
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import joblib

In [12]:
def extract_lbp(image, P=8, R=1):
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  lbp = local_binary_pattern(gray, P, R, method='uniform')
  hist, _ = np.histogram(lbp.ravel(), bins=P + 2, range=(0, P + 2))
  hist = hist.astype("float")
  hist /= (hist.sum() + 1e-7)
  return hist

def extract_hog(image):
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  h = hog(
    gray,
    orientations=9,
    pixels_per_cell=(8, 8),
    cells_per_block=(2, 2),
    block_norm='L2-Hys',
    visualize=False
  )
  return h

def extract_features(image):
  lbp_feat = extract_lbp(image)
  hog_feat = extract_hog(image)
  return np.hstack([lbp_feat, hog_feat])

In [13]:
IMG_SIZE = (128, 128)

DATASET_PATH = "../dataset/train"

X = []
y = []

for label in os.listdir(DATASET_PATH):
  class_dir = os.path.join(DATASET_PATH, label)
  if not os.path.isdir(class_dir):
    continue

  for fname in os.listdir(class_dir):
    img_path = os.path.join(class_dir, fname)
    img = cv2.imread(img_path)

    if img is None:
      continue

    # ==== Tambahan penting ====
    img = cv2.resize(img, IMG_SIZE)
    # ==========================

    feat = extract_features(img)
    X.append(feat)
    y.append(label)

X = np.array(X, dtype=object)  # sementara biarkan object dulu

In [14]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [15]:
model = LogisticRegression(
  multi_class='ovr',
  max_iter=3000
)

model.fit(X_scaled, y)
print("Training selesai!")

c:\laragon\www\pcd-coin-counting\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Training selesai!


In [16]:
preds = model.predict(X_scaled)
print(classification_report(y, preds))

              precision    recall  f1-score   support

         100       1.00      1.00      1.00        28
        1000       1.00      1.00      1.00        28
         200       1.00      1.00      1.00        28
          50       1.00      1.00      1.00        28
         500       1.00      1.00      1.00        28

    accuracy                           1.00       140
   macro avg       1.00      1.00      1.00       140
weighted avg       1.00      1.00      1.00       140



In [17]:
os.makedirs("../models", exist_ok=True)

joblib.dump(model, "../models/logistic_regression.pkl")
joblib.dump(scaler, "../models/scaler.pkl")

print("Model dan scaler berhasil disimpan.")

Model dan scaler berhasil disimpan.
